Importing necessary libraries for data preprosessing, visualising

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import warnings
%matplotlib inline


In [3]:
# importing dataset and printing first 5 rows
df = pd.read_csv(r'C:\Users\ASUS\Documents\Python\FYP\data.csv' , encoding='latin-1')
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
# naming the column for later process and printing column names
df.columns=[ "target","ids","date", "flag","user","text"]
df.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [5]:
# this funvti removes pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [6]:
# passing the text column into the function and saving into new column 'clean_tweet'
# printing the first 5 rows
df['clean_tweet'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")
df['clean_tweet'].head()

0    is upset that he can't update his Facebook by ...
1     I dived many times for the ball. Managed to s...
2      my whole body feels itchy and like its on fire 
3     no, it's not behaving at all. i'm mad. why am...
4                                  not the whole crew 
Name: clean_tweet, dtype: object

In [7]:
# Removing Punctuations, Numbers, and Special Characters
df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
df['clean_tweet'].head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12656\785911917.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")


0    is upset that he can t update his Facebook by ...
1     I dived many times for the ball  Managed to s...
2      my whole body feels itchy and like its on fire 
3     no  it s not behaving at all  i m mad  why am...
4                                  not the whole crew 
Name: clean_tweet, dtype: object

In [8]:
# remove short words
# For example, terms like “hmm”, “oh” are of very little use. It is better to get rid of them.
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df['clean_tweet'].head()

0    upset that update Facebook texting might resul...
1       dived many times ball Managed save rest bounds
2                     whole body feels itchy like fire
3                     behaving here because over there
4                                           whole crew
Name: clean_tweet, dtype: object

In [9]:
# Now we will tokenize all the cleaned tweets in our dataset. 
# Tokens are individual terms or words, and tokenization 
# is the process of splitting a string of text into tokens.


tokenized_tweet = df['clean_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [upset, that, update, Facebook, texting, might...
1    [dived, many, times, ball, Managed, save, rest...
2              [whole, body, feels, itchy, like, fire]
3               [behaving, here, because, over, there]
4                                        [whole, crew]
Name: clean_tweet, dtype: object

In [10]:
# Stemming is a rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc) from a word
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
tokenized_tweet.head()

0    [upset, that, updat, facebook, text, might, re...
1    [dive, mani, time, ball, manag, save, rest, bo...
2               [whole, bodi, feel, itchi, like, fire]
3                   [behav, here, becaus, over, there]
4                                        [whole, crew]
Name: clean_tweet, dtype: object

In [11]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
df['clean_tweet'] = tokenized_tweet
df['clean_tweet'].head()

0    upset that updat facebook text might result sc...
1            dive mani time ball manag save rest bound
2                      whole bodi feel itchi like fire
3                         behav here becaus over there
4                                           whole crew
Name: clean_tweet, dtype: object

Loading Word2Vec Model

In [12]:
from gensim.models import Word2Vec

Splitting Dataset for visualising both positive and negative labelled words.

In [13]:
df_pos= df[df['target']==0]
df_neg= df[df['target']==4]

In [14]:
tokenized_tweet_pos = df_pos['clean_tweet'].apply(lambda x: x.split()) # tokenizing positive tweets
tokenized_tweet_neg = df_neg['clean_tweet'].apply(lambda x: x.split()) # tokenizing negative tweets


Training Word2Vec

In [15]:
model_w2v_neg = Word2Vec(tokenized_tweet_neg, vector_size=200,  window=5)
model_w2v_pos = Word2Vec(tokenized_tweet_pos, vector_size=200,  window=5)

model_w2v_neg.train(tokenized_tweet_neg, total_examples= len(df_neg['clean_tweet']), epochs=20)
model_w2v_pos.train(tokenized_tweet_pos, total_examples= len(df_pos['clean_tweet']), epochs=20)


(95148903, 111486760)

In [16]:
#putting model_w2v_pos and model_w2v_neg list variable for visualising purpose
pos_words = list(model_w2v_pos.wv.key_to_index)
neg_words = list(model_w2v_neg.wv.key_to_index)

PCA

In [17]:
from sklearn.decomposition import PCA
X = model_w2v_pos.wv[model_w2v_pos.wv.key_to_index]
Y = model_w2v_neg.wv[model_w2v_neg.wv.key_to_index]
pca = PCA(n_components=2)
result_pos = pca.fit_transform(X)
result_neg = pca.fit_transform(Y)

#create df from the pca results
pca_df_pos = pd.DataFrame(result_pos, columns = ['x','y'])
pca_df_neg = pd.DataFrame(result_neg, columns = ['x','y'])


Visualising positive words as Blue dots and negative words as Orange dots

In [18]:
#Visualising with Plotly
import plotly.graph_objects as go
N = 1000000
# tokenized_tweet = list(model_w2v.wv.key_to_index)
fig1 = go.Figure(data=go.Scattergl(
   x = pca_df_pos['x'],
   y = pca_df_pos['y'],
   mode='markers',
   marker=dict(
       color='blue',
       colorscale='Viridis',
       line_width=1
   ),
   textposition="bottom center"
))

fig2 = go.Figure(data=go.Scattergl(
   x = pca_df_neg['x'],
   y = pca_df_neg['y'],
   mode='markers',
   marker=dict(
       color= '#FFA15A',
       colorscale='Viridis',
       line_width=1
   ),
   textposition="bottom center"
))
fig = go.Figure(data = fig1.data + fig2.data)

fig.show()